In [66]:
import openpyxl
import os
import pandas as pd

wb = openpyxl.load_workbook(os.path.expanduser('~/Desktop/celseq_spreadsheets/G000287_plate_layouts___Jacqui.xlsx'))

plate = 'LCE660'
sheet = wb[plate]

# dimentions of the plate
WELL_START_ROW = 9
WELL_START_COL = 'B'
WELL_END_ROW = 24
WELL_END_COL = 'Y'

# check if the sort description cell and well cells are in the right place
SORT_CELL = 'A27'
WELL_A_CELL = 'A9'
WELL_1_CELL = 'B8'

assert sheet[SORT_CELL].value.lower().startswith('sort')
assert sheet[WELL_A_CELL].value.lower() == 'a'
assert int(sheet[WELL_1_CELL].value) == 1

In [67]:
sample_dict = {}

# start with the two test samples
sample_dict[sheet['A28'].fill.start_color.index] = sheet['B28'].value
sample_dict[sheet['A29'].fill.start_color.index] = sheet['B29'].value

# the next two rows are usually blank, and then we process the other samples
row = 32
while(True):
    colour_cell = 'A' + str(row)
    sample_cell = 'B' + str(row)
    row += 1

    if sheet[colour_cell].fill.start_color.index == '00000000':
        break
    
    sample_dict[sheet[colour_cell].fill.start_color.index] = sheet[sample_cell].value

print(sample_dict)

{'FF0070C0': 'HRP Test before sort', 'FF00B0F0': 'HRP Test after sort', 'FFFFC000': 'NKC079-P4', 'FF00B050': 'NKC079-P5', 'FFFF66FF': 'NKC079-P6', 'FF66CCFF': 'NKC079-P7', 5: 'NKC REF-1 Control'}


In [68]:
sample_list = []
# iterate through the plate cells and extract sample name for each cell from the colour
columns = [chr(i) for i in range(ord('B'), ord('Y') + 1)]
for row in range(WELL_START_ROW, WELL_END_ROW + 1):
    for col in columns:
        cell_loc = col + str(row)
        cell = sheet[cell_loc]
        samplename = 'removed'
        if cell.fill.start_color.index != '00000000':
            cell_colour = cell.fill.start_color.index
            assert cell_colour in sample_dict
            samplename = sample_dict[cell_colour]
        
        # get well ID
        well_row = sheet['A' + str(row)].value
        well_col = sheet[col + str(8)].value
        well_id = well_row + str(well_col)

        sample_list.append((well_id, samplename))

sample_list

[('A1', 'HRP Test before sort'),
 ('A2', 'HRP Test after sort'),
 ('A3', 'NKC079-P4'),
 ('A4', 'NKC079-P4'),
 ('A5', 'NKC079-P4'),
 ('A6', 'NKC079-P6'),
 ('A7', 'NKC079-P6'),
 ('A8', 'NKC079-P6'),
 ('A9', 'NKC079-P6'),
 ('A10', 'NKC079-P6'),
 ('A11', 'NKC079-P6'),
 ('A12', 'NKC079-P6'),
 ('A13', 'NKC079-P7'),
 ('A14', 'NKC079-P7'),
 ('A15', 'NKC079-P7'),
 ('A16', 'NKC079-P7'),
 ('A17', 'NKC079-P7'),
 ('A18', 'NKC079-P7'),
 ('A19', 'NKC079-P7'),
 ('A20', 'NKC079-P5'),
 ('A21', 'NKC079-P5'),
 ('A22', 'NKC079-P5'),
 ('A23', 'HRP Test after sort'),
 ('A24', 'HRP Test before sort'),
 ('B1', 'NKC079-P4'),
 ('B2', 'NKC079-P4'),
 ('B3', 'NKC079-P4'),
 ('B4', 'NKC079-P4'),
 ('B5', 'NKC079-P4'),
 ('B6', 'NKC079-P6'),
 ('B7', 'NKC079-P6'),
 ('B8', 'NKC079-P6'),
 ('B9', 'NKC079-P6'),
 ('B10', 'NKC079-P6'),
 ('B11', 'NKC079-P6'),
 ('B12', 'NKC079-P6'),
 ('B13', 'NKC079-P7'),
 ('B14', 'NKC079-P7'),
 ('B15', 'NKC079-P7'),
 ('B16', 'NKC079-P7'),
 ('B17', 'NKC079-P7'),
 ('B18', 'NKC079-P7'),
 ('B19', '

In [69]:
samplesheet = pd.DataFrame(sample_list, columns=['well_position', 'sample'])
samplesheet['plate'] = plate
samplesheet.head()

,well_position,sample,plate
0,A1,HRP Test before sort,LCE660
1,A2,HRP Test after sort,LCE660
2,A3,NKC079-P4,LCE660
3,A4,NKC079-P4,LCE660
4,A5,NKC079-P4,LCE660


In [70]:
samplesheet.to_csv(os.path.expanduser('~/Desktop/celseq_spreadsheets/samplesheet.csv'), index=False)

In [71]:
cell = sheet.cell(row=27, column=1)
cell.coordinate

'A27'

In [82]:
WELL_ROWS = 16
WELL_COLS = 24 

# how far we search for the start of the well positions and sample lookup
ROWMAX = 100
COLMAX = 25

def find_sample_start_cell(sheet):
    '''
    Find the start of the sample names in the spreadsheet.
    Return the cell.
    '''
    sample_start = None

    for col in range(1, COLMAX):
        for row in range(1, ROWMAX):
            cell = sheet.cell(row=row, column=col)
            if cell.value and str(cell.value).lower() in ['sort description', 'sort discription']:
                sample_start = cell
                break

    return sample_start

sample_start_cell = find_sample_start_cell(sheet)
assert sample_start_cell, 'Could not find sample start cell in sheet {}'.format(sheet.title)

sample_dict = {}

current_cell = sample_start_cell
tolerate_n_blank_rows = 2
while(True):
    current_cell = sheet.cell(row=current_cell.row + 1, column=current_cell.column)
    colour = sheet.cell(row=current_cell.row, column=current_cell.column).fill.start_color.index

    if colour == '00000000':
        if tolerate_n_blank_rows == 0:
            break
        tolerate_n_blank_rows -= 1
        continue

    value = sheet.cell(current_cell.row, current_cell.column + 1).value
    sample_dict[colour] = value

sample_dict

{'FF0070C0': 'HRP Test before sort',
 'FF00B0F0': 'HRP Test after sort',
 'FFFFC000': 'NKC079-P4',
 'FF00B050': 'NKC079-P5',
 'FFFF66FF': 'NKC079-P6',
 'FF66CCFF': 'NKC079-P7',
 5: 'NKC REF-1 Control'}